In [ ]:
import xarray as xr
!pip install wget
import numpy as np
!pip install e2cnn
import e2cnn
import torch
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
!pip install torchsummary
from torchsummary import summary
from util import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Load dataset

Let's get some data to play with. \
I have made available for download some data, using the function below, but it would be more fun to upload your own data.

In [ ]:
ds = get_data()

In [ ]:
ds

For those uploading their own data: \
Make sure you have O(10) timesteps and preferably multiple vertical levels of at least one 2-d vector in the horizontal plane and preferably one 'scalar' (which could be vertical velocity component) as inputs. Let's start with just one output variable. \
Data should be colocated, so some de-staggering may be in order.

For those who want to use my data: \
While waiting for others to upload, please try to debug the get_data() function in util.py which currently needs to use wget though I feel xr.open_dataset should suffice. \
Or, modify the code below to use xarray, or any other library you think is useful, instead of numpy.

## Brief preproccessing

First, let's reorder the time index as our sample index and our z-index as a channel index for 2-d convolution.

My data's index order was originally (z,y,x,t) so the details of the transpose might change if using your own data. \
Index order should be (sample, channel=vertical level, x, y)

In [ ]:
Ustd=np.std(np.sqrt(ds['u'].values**2+ds['v'].values**2))

u=np.transpose(scale(ds['u'].values,std=Ustd), [3,0,1,2]) 
v=np.transpose(scale(ds['v'].values,std=Ustd), [3,0,1,2])
w=np.transpose(scale(ds['w'].values) , [3,0,1,2])
tau12=np.transpose(scale(ds['tau12'].values), [3,0,1,2])
nz=u.shape[1] # might need to change index for your data

In the next step we will concatenate the input variables along the channel dimension.
At this point, we lose the information that the two components of the vector are related to each other geometrically.


In [ ]:
inputFields = np.concatenate((u, v, w),axis=1)
outputFields = tau12

## A pytorch baseline

Let's make a simple pytorch CNN to compare to an equivariant CNN, which we will construct later. 

In [ ]:
class pytorchCNN(torch.nn.Module):
    def __init__(self, input_shape, nz):
        super(pytorchCNN, self).__init__()
        
        C=input_shape[1]*np.array([2,2]) # number of channel in hidden layers
        
        self.conv1 = torch.nn.Conv2d(input_shape[1], C[0], kernel_size=3,padding=1,padding_mode='circular')
        self.relu1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(C[0], C[1], kernel_size=3,padding=1,padding_mode='circular')
        self.relu2 = torch.nn.ReLU()
        self.conv3 = torch.nn.Conv2d(C[1], nz, kernel_size=3,padding=1,padding_mode='circular')
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        
        return x

Let's split our data into training and validation, uisng the mask below. 

In [ ]:
mask =  np.random.rand(inputFields.shape[0]) < 0.80

In addition to splitting the data, the next cell will convert to the appropriate data types.

In [ ]:
x_train, x_test = torch.from_numpy(inputFields[mask]).float().to(device), torch.from_numpy(inputFields[~mask]).float().to(device)
y_train, y_test = torch.from_numpy(outputFields[mask]).float().to(device), torch.from_numpy(outputFields[~mask]).float().to(device)

Instantiating our model below.

In [ ]:
model=pytorchCNN(x_train.shape,nz).float().to(device)

Some reasonable choices for optimizer and loss function below. Feel free to change.

In [ ]:
optimizer = torch.optim.Adam(model.parameters())#,lr=0.001)
criterion = torch.nn.MSELoss() # MSE loss function
validation_loss = list()
train_loss = list()

Training loop, nothing fancy. \
If you want the loss to be printed after every epoch, uncomment the commented lines and comment the two below those.

In [ ]:
n_epochs = 200 #Number of epochs
for epoch in range(n_epochs):
    train_model(model,x_train,y_train,criterion,optimizer)
    # train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, 'train'))
    # validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer))
    train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, None))
    validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer,None))


Loss curves below don't have to be perfect, we just need some reasonable level of performance so we can make sense of the contour plots below.

In [ ]:
plot_losses(validation_loss,train_loss)
plot_losses(validation_loss,train_loss,startEpoch=len(validation_loss)-10)

Now, we plot some predictions from the validation data.

In [ ]:
prediction=model(x_test)

In [ ]:
kplot=nz//2
tplot=y_test.shape[0]//2

plot_contour(y_test,prediction,tplot,kplot)

To reiterate, the performance does not need to be optimized, because the focus will be on how the model does on unrotated vs rotated data.

Now, we need some rotated data! \
I wrote the function, myrotate, specifically for rotating the velocity vector, ordered as below (u,v,w) and the outputField, tau_12. Tau_12 doesn't rotate exactly like a scalar, if your output does, delete the input, out_type, in the myrotate() call. If it still doesn't work for you, don't worry you'll get a chance to write your own rotate function later.

In [ ]:
inputFields_rotated,outputFields_rotated = myrotate(np.stack((u,v,w)),outputFields,krot=1,out_type='sign')

Because rotating didn't affect the time/sample dimension, we'll use the same mask as during training.

In [ ]:
x_train_rotated, x_test_rotated = torch.from_numpy(inputFields_rotated[mask]).float().to(device), torch.from_numpy(inputFields_rotated[~mask]).float().to(device)
y_train_rotated, y_test_rotated = torch.from_numpy(outputFields_rotated[mask]).float().to(device), torch.from_numpy(outputFields_rotated[~mask]).float().to(device)

The following plots are just to establish that our rotations work. If you didn't delete the out_type=sign input to myrotate() and notice and sign error in the bottom plot, try deleting that now.

In [ ]:
#plot_quiver( [x_test[:,0:nz],x_test[:,nz:2*nz]],[x_test_rotated[:,0:nz],x_test_rotated[:,nz:2*nz]],tplot,kplot)
plot_contour(x_test[:,0:nz],x_test_rotated[:,nz:2*nz],tplot,kplot,text=['Original, u','Rotated, v'])
plot_contour(x_test[:,nz:2*nz],-x_test_rotated[:,0:nz],tplot,kplot,text=['Original, v','Rotated, -u'])
plot_contour(x_test[:,-nz:],x_test_rotated[:,-nz:],tplot,kplot,text=['Original, w','Rotated, w'])
plot_contour(y_test,-y_test_rotated,tplot,kplot,text=['Original, tau_12','Rotated, -tau_12'],clim_mode='share')

If your inputs and outputs are rotating correctly, time to test our prediction on roated inputs.

In [ ]:
prediction_rotated=model(x_test_rotated)

Again, there is a negative sign below relating to the type of tau_12, modify if necessary for your data.

In [ ]:
plot_contour(y_test,prediction,tplot,kplot)
plot_contour(-y_test_rotated,-prediction_rotated,tplot,kplot)

The take-away here is that the pytorch baseline, albeit without data augmentation, has absolutely no predictive skill when ingesting rotated input data.

## e2cnn version

Compare the pytorch code, pasted below for convenience, to the e2cnn version of the same network.

In [ ]:
class pytorchCNN(torch.nn.Module):
    def __init__(self, input_shape, nz):
        super(pytorchCNN, self).__init__()
        
        C=input_shape[1]*np.array([2,2]) # number of channel in hidden layers
        
        self.conv1 = torch.nn.Conv2d(input_shape[1], C[0], kernel_size=3,padding=1,padding_mode='circular')
        self.relu1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(C[0], C[1], kernel_size=3,padding=1,padding_mode='circular')
        self.relu2 = torch.nn.ReLU()
        self.conv3 = torch.nn.Conv2d(C[1], nz, kernel_size=3,padding=1,padding_mode='circular')
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        
        return x

First, note the forward() routines look almost identical. The only difference is that I convert the input tensor, x, to the e2cnn data type, GeometricTensor. This is basically a fancy wrapper around the torch Tensor data type, which adds the geometric information. \
Where does this geometric information come from? That is encoded in self.type_in which I define with e2cnn.nn.FieldType during the init. Indeed, defining these FieldType objects is most of the extra code in the e2cnn init compared to that of the pytorch baseline. \
In the calls to R2Conv, another fancy wrapper around pytorch's Conv2d, the FieldType plays the role of the channels in and channels out.

Other than defining the dimensionality of the convolutions, the e2cnn.nn.FieldType objects says with what kind of 'representation' each feature is associated. The represention of the transformations of a particular input have a certain 'irrep' which may correspond to familiar scalar, irrep(0), and vector, irrep(1), inputs. The output is associated with the less familiar 'sign' representation, irrep(2), discussed in the presentation.

In [ ]:
class e2cnnCNN(torch.nn.Module):
    def __init__(self, input_shape, nz):
        super(e2cnnCNN, self).__init__()
        
        C=input_shape[1]*np.array([2,2]) # number of channel in hidden layers
        
        C4group = e2cnn.gspaces.Rot2dOnR2(N = 4)
        self.type_in = e2cnn.nn.FieldType(C4group, nz*[C4group.irrep(1)]+nz*[C4group.trivial_repr])
        self.type_hid1 = e2cnn.nn.FieldType(C4group, C[0]*[C4group.regular_repr])
        self.type_hid2 = e2cnn.nn.FieldType(C4group, C[1]*[C4group.regular_repr])
        self.type_out = e2cnn.nn.FieldType(C4group, nz*[C4group.irrep(2)])
        
        
        self.conv1 = e2cnn.nn.R2Conv(self.type_in, self.type_hid1,  kernel_size=3,padding=1,padding_mode='circular')
        self.relu1 = e2cnn.nn.ReLU(self.type_hid1)
        self.conv2 = e2cnn.nn.R2Conv(self.type_hid1, self.type_hid2,  kernel_size=3,padding=1,padding_mode='circular')
        self.relu2 = e2cnn.nn.ReLU(self.type_hid2)
        self.conv3 = e2cnn.nn.R2Conv(self.type_hid2, self.type_out, kernel_size=3,padding=1,padding_mode='circular')
        
    def forward(self,x):
        
        x = e2cnn.nn.GeometricTensor(x, self.type_in)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
      
        return x.tensor
        

The hidden layers have the 'regular' representation, which for the cyclic group just corresponds to all the cyclic permutations. Kind of confusing, but easy to type!

In [ ]:
N=4
C4group = e2cnn.gspaces.Rot2dOnR2(N = N)
x=torch.from_numpy(np.random.rand(1,N,1,1))
x=e2cnn.nn.GeometricTensor(x, e2cnn.nn.FieldType(C4group, [C4group.regular_repr]))
for g in C4group.testing_elements:
    # Matthew 20:16
    print('Rotation by '+str(g*90)+' degrees is '+str(g)+' cyclic permutations for regular representation')
    print(x.transform(g))

In [ ]:
x=torch.from_numpy(np.random.rand(1,1,1,1))
x=e2cnn.nn.GeometricTensor(x, e2cnn.nn.FieldType(C4group, [C4group.trivial_repr]))
for g in C4group.testing_elements:
    # Matthew 20:16
    print('Rotation by '+str(g*90)+' degrees, no effect for trivial representation')
    print(x.transform(g))

In [ ]:
x=torch.from_numpy(np.random.rand(1,1,1,1))
x=e2cnn.nn.GeometricTensor(x, e2cnn.nn.FieldType(C4group, [C4group.irrep(2)]))
for g in C4group.testing_elements:
    # Matthew 20:16
    print('Rotation by '+str(g*90)+' degrees = '+str(g)+' sign changes for sign representation')
    print(x.transform(g))

In [ ]:
x=torch.from_numpy(np.random.rand(1,2,1,1))
x=e2cnn.nn.GeometricTensor(x, e2cnn.nn.FieldType(C4group, [C4group.irrep(1)]))
for g in C4group.testing_elements:
    # Matthew 20:16
    print('Rotation by '+str(g*90)+' degrees = rotation as for vector for irrep(1)')
    print(x.transform(g))

Note, when we combined our input variables in the most natural way: \
 &emsp;  inputFields = np.concatenate((u, v, w),axis=1) \
The resulting order of each sample was \
\[u(k=0), u(k=1), u(k=2),..., u(k=# vert levels - 1), v(k=0), v(k=1), v(k=2),..., v(k=# vertical levels - 1)\] \
In order for the e2cnn library to associate the u,v variables as components of the same vector, we need to instead have \
 \[u(k=0),  v(k=0), u(k=1), v(k=1), u(k=2), v(k=2),..., u(k=# vert levels - 1), v(k=# vertical levels - 1)\] \

In [ ]:
def reshape_input(inputFields_in,nz):
    #Resulting order is
    #u(k=0),v(k=0),u(k=1),v(k=1),u(k=2),v(k=2),...,w(k=0),w(k=1),w(k=2),....
    inputFields_out=inputFields_in.copy()
    for v in range(2):
        for k in range(nz):
            inputFields_out[:,2*k+v,:,:]=inputFields_in[:,v*nz+k,:,:]

    return inputFields_out

In [ ]:
inputFields = reshape_input(np.concatenate((u, v, w),axis=1),nz)
outputFields = tau12

Apart from the reshaping explained above, the following cells proceed just as we did with pytorch before. \
Note, even the loss and optimizers are being set with the torch.nn and torch.optim modules, not e2cnn.nn and e2nn.optim (which doesn't exist).

In [ ]:
x_train, x_test = torch.from_numpy(inputFields[mask]).float().to(device), torch.from_numpy(inputFields[~mask]).float().to(device)
y_train, y_test = torch.from_numpy(outputFields[mask]).float().to(device), torch.from_numpy(outputFields[~mask]).float().to(device)

In [ ]:
model=e2cnnCNN(x_train.shape, nz).float().to(device)

In [ ]:
baseline=pytorchCNN(x_train.shape,nz).float().to(device)
summary(baseline,x_train.shape[1:])
summary(model,x_train.shape[1:])

In [ ]:
optimizer = torch.optim.Adam(model.parameters())#,lr=0.001)
criterion = torch.nn.MSELoss() # MSE loss function
validation_loss = list()
train_loss = list()

In [ ]:
n_epochs = 400 #Number of epochs
for epoch in range(n_epochs):
    train_model(model,x_train,y_train,criterion,optimizer)
    # train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, 'train'))
    # validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer))
    train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, None))
    validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer,None))


One small difference worth noting is that the e2cnn does seem to take longer to train than the pytorch. For instance with my data, we are arguable overfitting after 200 epochs for the pytorch model, but the loss curve of the e2cnn is still smoothly decreasing after 400 epochs. 

In [ ]:
plot_losses(validation_loss,train_loss)
plot_losses(validation_loss,train_loss,startEpoch=len(validation_loss)-10)

In [ ]:
prediction=model(x_test)

In [ ]:
# Uncomment to assess if some hyperparameter tuning is necessary for reasonable performance if using your own data
# kplot=nz//2
# tplot=y_test.shape[0]//2
# plot_contour(y_test,prediction,tplot,kplot)

As before, we want to compare predictions made from unrotated and rotated inputs, so we have to rotate the data. 

In [ ]:
inputFields_rotated,outputFields_rotated = myrotate(np.stack((u,v,w)),outputFields,krot=1,out_type='sign')
inputFields_rotated=reshape_input(inputFields_rotated,nz)

In [ ]:
x_train_rotated, x_test_rotated = torch.from_numpy(inputFields_rotated[mask]).float().to(device), torch.from_numpy(inputFields_rotated[~mask]).float().to(device)
y_train_rotated, y_test_rotated = torch.from_numpy(outputFields_rotated[mask]).float().to(device), torch.from_numpy(outputFields_rotated[~mask]).float().to(device)

In [ ]:
# Uncomment and possibly modify if need to convince yourself myrotate() is working as intended
# plot_contour(x_test[:,0:2*nz:2],x_test_rotated[:,1:2*nz+1:2],tplot,kplot,text=['Original, u','Rotated, v'])
# plot_contour(x_test[:,-nz:],x_test_rotated[:,-nz:],tplot,kplot,text=['Original, w','Rotated, w'])
# plot_contour(y_test,-y_test_rotated,tplot,kplot,text=['Original, tau_12','Rotated, -tau_12'])

In [ ]:
prediction_rotated=model(x_test_rotated)

In [ ]:
plot_contour(y_test,prediction,tplot,kplot)
plot_contour(-y_test_rotated,-prediction_rotated,tplot,kplot,text=["Rotated, true","Predicted from rotated input"])

Unlike, pytorch, the rotated input leads to rotated output from the e2cnn version which is precisely what we would obtain if we simply rotated after prediction from the original data. \
Equivariance!

## Coding activity

In my example, we took the 3 components of velocity and predicted 1 component, tau_12, of the momentum flux. \
Let's write a new model which will take the same input but instead output the 3 shear components of momentum flux, tau_12, tau_13, and tau_23. Try and translate this problem to your own data if necessary, e.g. if you looked at vertical buoyancy flux, <w'b'> try to include <u'b'> and <v'b'>. 
You will also need to modify the model definition. \
Based on your choices of output, you will need to modify the model definition:

In [ ]:
class e2cnnCNN(torch.nn.Module):
    def __init__(self, input_shape, nz):
        super(e2cnnCNN, self).__init__()
        
       # ???
        
    def forward(self,x):
        
        x = e2cnn.nn.GeometricTensor(x, self.type_in)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
      
        return x.tensor
        

Because the additional components form a vector in the horizontal planes, \[tau_13, tau_23\] (or \[<u'b'>, <v'b'>\], etc.), you may need to define a reshape_output, similar to my reshape_input function, which takes np.concatenate((tau_12,tau_13,tau_23), axis=1) as an input and outputs the same variables but reordered as required by the e2cnn.FieldType for the output layer, which you will need to define above.

In [ ]:
def reshape_output(outputFields_in,nz):
    #Resulting order is ???

    return outputFields_out

As with horizontal velocity vector, we need to use the same scaling for each component of the output vectors. I have gone ahead and done that for the example with my data, modify as necessary for your own data.

In [ ]:
taui3std = np.std(np.sqrt(ds['tau13'].values**2+ds['tau23'].values**2))

#tau12=np.transpose(scale(ds['tau12'].values,mean=0), [3,0,1,2])
tau13=np.transpose(scale(ds['tau13'].values,mean=0,std=taui3std), [3,0,1,2])
tau23=np.transpose(scale(ds['tau23'].values,mean=0,std=taui3std), [3,0,1,2])

outputFields = reshape_output(np.concatenate((tau12,tau13,tau23), axis=1),nz)

In [ ]:
x_train, x_test = torch.from_numpy(inputFields[mask]).float().to(device), torch.from_numpy(inputFields[~mask]).float().to(device)
y_train, y_test = torch.from_numpy(outputFields[mask]).float().to(device), torch.from_numpy(outputFields[~mask]).float().to(device)

In [ ]:
model=e2cnnCNN(x_train.shape,nz).float().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())#,lr=0.001)
criterion = torch.nn.MSELoss() # MSE loss function
validation_loss = list()
train_loss = list()

In [ ]:
n_epochs = 400 #Number of epochs
for epoch in range(n_epochs):
    train_model(model,x_train,y_train,criterion,optimizer)
    # train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, 'train'))
    # validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer))
    train_loss.append(test_model(model,x_train,y_train,criterion,optimizer, None))
    validation_loss.append(test_model(model,x_test,y_test,criterion,optimizer,None))


In [ ]:
plot_losses(validation_loss,train_loss,startEpoch=len(validation_loss)-10)

In [ ]:
prediction=model(x_test)

In [ ]:
kplot=nz//2
tplot=y_test.shape[0]//2
plot_contour(y_test[:,0:nz],prediction[:,0:nz],tplot,kplot)

As before, it will be nice to check the predictions against some manually rotated data.

In [ ]:
def rotate(inputFields_in,outputFields_in,krot,out_type='scalar'):
    inputFields_out=np.empty(inputFields_in.shape)
    outputFields_out=np.empty(outputFields_in.shape)
    
    # first move values to new location on grid
    
    # second rotate the vector, change sign rep, etc.

    # final shape should have variable concatenated along axis=1                  
    inputFields_final=np.concatenate([inputFields_out[i] for i in range(inputFields_out.shape[0])],axis=1)
    outputFields_final=np.concatenate([outputFields_out[i] for i in range(outputFields_out.shape[0])],axis=1)
    


    return inputFields_final,outputFields_final

In [ ]:
inputFields_rotated,outputFields_rotated = rotate(np.stack((u,v,w)),np.stack((tau12,tau13,tau23)),krot=1,out_type='sign')
inputFields_rotated=reshape_input(inputFields_rotated,nz)
outputFields_rotated=reshape_output(outputFields_rotated,nz)

In [ ]:
x_train_rotated, x_test_rotated = torch.from_numpy(inputFields_rotated[mask]).float().to(device), torch.from_numpy(inputFields_rotated[~mask]).float().to(device)
y_train_rotated, y_test_rotated = torch.from_numpy(outputFields_rotated[mask]).float().to(device), torch.from_numpy(outputFields_rotated[~mask]).float().to(device)

In [ ]:
nz=u.shape[1] # might need to change index for your data
plot_contour(x_test[:,0:2*nz:2],x_test_rotated[:,1:2*nz+1:2],tplot,kplot,text=['Original, u','Rotated, v'])
plot_contour(x_test[:,-nz:],x_test_rotated[:,-nz:],tplot,kplot,text=['Original, w','Rotated, w'])
plot_contour(y_test[:,nz:3*nz:2],y_test_rotated[:,nz+1:3*nz+1:2],tplot,kplot,text=['Original, tau_13','Rotated, tau_23'])
plot_contour(y_test[:,0:nz],-y_test_rotated[:,0:nz],tplot,kplot,text=['Original, tau_12','Rotated, -tau_12'])

In [ ]:
prediction_rotated=model(x_test_rotated)

In [ ]:
plot_contour(y_test[:,nz:3*nz:2],prediction[:,nz:3*nz:2],tplot,kplot)
plot_contour(y_test_rotated[:,nz+1:3*nz+1:2],prediction_rotated[:,nz+1:3*nz+1:2],tplot,kplot)

In [ ]:
plot_quiver([y_test[:,nz:3*nz:2],y_test[:,nz+1:3*nz+1:2]],[prediction[:,nz:3*nz:2],prediction[:,nz+1:3*nz+1:2]],tplot,kplot)
plot_quiver([y_test_rotated[:,nz:3*nz:2],y_test_rotated[:,nz+1:3*nz+1:2]],[prediction_rotated[:,nz:3*nz:2],prediction_rotated[:,nz+1:3*nz+1:2]],tplot,kplot,text=["True, Rotated", "Predicted, Rotated"])